# Differential Equation Solving with SciPy

This notebook demonstrates the differences in differential equation solving techniques offered by the Python [SciPy](scipy.org) package. There are two primary approaches in SciPy:
1. The older `scipy.integrate.odeint` function
2. The newer `scipy.integrate.solve_ivp` function

## Getting started

To begin, we should mention that there is an accessible and comprehensive Python math text from UCBerkeley called [Python Numerical Methods](https://pythonnumericalmethods.studentorg.berkeley.edu/notebooks/Index.html). Chapter 22 is called "[Ordinary Differential Equation - Initial Value Problems](https://pythonnumericalmethods.studentorg.berkeley.edu/notebooks/chapter22.00-ODE-Initial-Value-Problems.html)," and it might be pointed out that the difference between the two is that Initial Value Problems supply starting conditions to the Ordinary Differential Equations so that they can be solved with a unique numerical functions.

## The Lorenz System

This example is adopted from the book chapter's [Problems](https://pythonnumericalmethods.studentorg.berkeley.edu/notebooks/chapter22.08-Summary-and-Problems.html#problems) section.

The Lorenz system is a set of three coupled, nonlinear differential equations:

$$\begin{align}
\frac{dx}{dt} &= \sigma(y - x) \\
\frac{dy}{dt} &= x(\rho - z) - y \\
\frac{dz}{dt} &= xy - \beta z
\end{align}$$

Where:
- $\sigma$ (sigma) is the Prandtl number
- $\rho$ (rho) is the Rayleigh number
- $\beta$ (beta) is related to the physical dimensions of the system

The classic parameter values are $\sigma = 10$, $\rho = 28$, and $\beta = 8/3$, which produce the famous "butterfly" strange attractor.

We will set up our system and conditions below. Also, we'll put this here: 🦋

In [2]:
sigma = 10
rho = 28
beta = 8/3

def lorenz(S, t):
    x, y, z = S
    dxdt = sigma * (y - x)
    dydt = x * (rho - z) - y
    dzdt = x * y - beta * z
    return [dxdt, dydt, dzdt]

# Initial conditions
S0 = [1.0, 0.0, 0.0]  # classic starting point

# Discretize time
t = np.linspace(0, 40, 10000)

In [3]:
# We will almost certainly need these three standards:

import numpy as np
from scipy.integrate import odeint
import matplotlib.pyplot as plt



## `odeint`

Starting with `odeint`, the older of the two solutions scipy offers, we might point out that the package always operates using the LSODA solver written in FORTRAN at [Lawrence Livermore NL](https://computing.llnl.gov/projects/odepack). The L in LSODA stands for Livermore, in fact, a bit of trivia that can be gleaned from the [excellent README file](https://github.com/scipy/scipy/tree/main/scipy/integrate/odepack) for ODEPACK solvers at the scipy repo.

Let's work with Lorenz in `odeint`.

In [4]:


S, info = odeint(lorenz, state0, t)

print("Solution shape:", sol.shape)
print("Solution:", sol)
print("Info:", info)


ValueError: too many values to unpack (expected 2)

### Notes


### References

Kong, Q., Siauw, T., & Bayen, A. M. (2021). *Python Programming And Numerical Methods: A Guide For Engineers And Scientists*. Academic Press. Available online at: https://pythonnumericalmethods.studentorg.berkeley.edu/notebooks/

Lorenz, E. N. (1963). Deterministic nonperiodic flow. *Journal of the Atmospheric Sciences*, 20(2), 130-141.

SciPy Documentation. (n.d.). Integration and ODEs (`scipy.integrate`). Retrieved from https://docs.scipy.org/doc/scipy/reference/tutorial/integrate.html